In [13]:
!pip install dash

In [14]:
import pandas as pd
import dash
df=pd.read_csv('dfJoinAdd3.csv')
df.head(1)

<ipython-input-14-2ad59f68c2a5>:3: DtypeWarning:

Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.



,RideID,OrigDest_x,Lat1,Lng1,OrigDest_y,Lat2,Lng2,Distancia,Dia,Price,schedule_time,ProductID
0,1183200,1,-26.329754,-48.840428,2,-26.2554657,-48.6434197,21.327087,1,89.0,10:09:45.628170,Flash


In [16]:
from dash import dcc, html, Input, Output
import plotly.express as px
from collections import OrderedDict

# Mapeamento dos dias da semana
mapa_dias = OrderedDict({
    1: 'Segunda',
    2: 'Terça',
    3: 'Quarta',
    4: 'Quinta',
    5: 'Sexta',
    6: 'Sábado',
    7: 'Domingo'
})
df['DiaNome'] = df['Dia'].map(mapa_dias)
df['DiaNome'] = pd.Categorical(df['DiaNome'], categories=list(mapa_dias.values()), ordered=True)

# Extrai hora da coluna schedule_time
df['Hora'] = pd.to_datetime(df['schedule_time']).dt.hour

# Inicializa o app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.Div([
        html.H1('Pick Your Driver'),
        html.P("")],
        style={'width': '75%','text-align': 'center', 'padding': '20px', 'backgroundColor': '#f8f9fa'}),

    # Filtros
    html.Div([
        html.Label("Selecione os dias da semana:"),
        dcc.Checklist(
            id='Dia-checkbox',
            options=[{'label': Dia, 'value': Dia} for Dia in mapa_dias.values()],
            value=['Segunda', 'Terça'],  # valor inicial
            inline=True
        ),
        html.Label("Selecione o tipo de gráfico:"),
        dcc.Dropdown(
            id='tipo-grafico',
            options=[
                {'label': 'Preço médio por dia', 'value': 'media_dia'},
                {'label': 'Preço por hora', 'value': 'preco_hora'},
                {'label': 'Distância vs Preço', 'value': 'distancia_preco'}
            ],
            value='media_dia'
        )
    ], style={'width': '75%','padding': '20px', 'backgroundColor': '#e9ecef'}),

    # Gráfico
    html.Div([
        dcc.Graph(id='hora-vs-Dia')
    ], style={'width': '75%','padding': '20px'})
])

# Callback
@app.callback(
    Output('hora-vs-Dia', 'figure'),
    [
        Input('Dia-checkbox', 'value'),
        Input('tipo-grafico', 'value')
    ]
)
def update_graph(dias_selecionados, tipo):
    df_filtrado = df[df['DiaNome'].isin(dias_selecionados)]

    if df_filtrado.empty:
        return px.scatter(title='Nenhum dado disponível para os dias selecionados.')

    if tipo == 'media_dia':
        fig = px.bar(df_filtrado.groupby('DiaNome')['Price'].mean().reset_index(),
                     x='DiaNome', y='Price',
                     title='Preço médio por dia da semana',
                     color='DiaNome')

    elif tipo == 'preco_hora':
        fig = px.box(df_filtrado, x='Hora', y='Price', color='DiaNome',
                     title='Distribuição de preço por horário')

    elif tipo == 'distancia_preco':
        df_500 = df_filtrado[df_filtrado['Distancia'] <= 500]
        fig = px.scatter(df_500, x='Distancia', y='Price', color='DiaNome', trendline='ols',
                         title='Distância (máx 500 km) vs Preço')

    else:
        fig = px.scatter(df_filtrado, x='DiaNome', y='Price',
                         title='Dia vs Preço',
                         color='DiaNome')

    return fig

if __name__ == '__main__':
    app.run(debug=True)

<ipython-input-16-b42b3cf19c13>:19: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



<IPython.core.display.Javascript object>